In [1]:
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
import time
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
btc_min_data = pd.read_csv('/Users/ashutoshbhawsar/Documents/Fall 2022 Courses/DIC/DIC project/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv') 

print(btc_min_data)

          Timestamp      Open      High       Low     Close  Volume_(BTC)  \
0        1325317920      4.39      4.39      4.39      4.39      0.455581   
1        1325317980       NaN       NaN       NaN       NaN           NaN   
2        1325318040       NaN       NaN       NaN       NaN           NaN   
3        1325318100       NaN       NaN       NaN       NaN           NaN   
4        1325318160       NaN       NaN       NaN       NaN           NaN   
...             ...       ...       ...       ...       ...           ...   
4857372  1617148560  58714.31  58714.31  58686.00  58686.00      1.384487   
4857373  1617148620  58683.97  58693.43  58683.97  58685.81      7.294848   
4857374  1617148680  58693.43  58723.84  58693.43  58723.84      1.705682   
4857375  1617148740  58742.18  58770.38  58742.18  58760.59      0.720415   
4857376  1617148800  58767.75  58778.18  58755.97  58778.18      2.712831   

         Volume_(Currency)  Weighted_Price  
0                 2.000000    

In [3]:

print(btc_min_data.columns)

Index(['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume_(BTC)',
       'Volume_(Currency)', 'Weighted_Price'],
      dtype='object')


In [4]:
# convert unix time to timestamp
btc_min_data['Timestamp']=pd.to_datetime(btc_min_data['Timestamp'],unit='s')

In [8]:
btc_min_data=btc_min_data.rename(columns={'Volume_(BTC)':"Volume_BTC", 'Volume_(Currency)':"Volume_Currency"})

In [6]:
# Drop all Not-a-number(NaN) or NA values
btc_min_data=btc_min_data.dropna()

In [7]:
# Temp convert timestamp to date
#btc_min_data['Timestamp']=pd.to_datetime(btc_min_data['Timestamp']).dt.date
#type(btc_min_data['Timestamp'])

/var/folders/nj/x9xgw62x63z_r3j8g2qtz4b00000gn/T/ipykernel_22327/1658431405.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btc_min_data['Timestamp']=pd.to_datetime(btc_min_data['Timestamp']).dt.date


In [9]:
btc_min_data

,Timestamp,Open,High,Low,Close,Volume_BTC,Volume_Currency,Weighted_Price
0,2011-12-31 07:52:00,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
478,2011-12-31 15:50:00,4.39,4.39,4.39,4.39,48.000000,210.720000,4.390000
547,2011-12-31 16:59:00,4.50,4.57,4.50,4.57,37.862297,171.380338,4.526411
548,2011-12-31 17:00:00,4.58,4.58,4.58,4.58,9.000000,41.220000,4.580000
1224,2012-01-01 04:16:00,4.58,4.58,4.58,4.58,1.502000,6.879160,4.580000
...,...,...,...,...,...,...,...,...
4857372,2021-03-30 23:56:00,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339
4857373,2021-03-30 23:57:00,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508
4857374,2021-03-30 23:58:00,58693.43,58723.84,58693.43,58723.84,1.705682,100117.070370,58696.198496
4857375,2021-03-30 23:59:00,58742.18,58770.38,58742.18,58760.59,0.720415,42332.958633,58761.866202


In [9]:
btc_min_data.groupby('Timestamp')['Low'].min()

Timestamp
2011-12-31        4.39
2012-01-01        4.58
2012-01-02        5.00
2012-01-03        5.14
2012-01-04        4.93
                ...   
2021-03-27    53948.35
2021-03-28    54677.51
2021-03-29    54892.42
2021-03-30    57011.00
2021-03-31    58755.97
Name: Low, Length: 3376, dtype: float64

In [10]:
df = pd.DataFrame({})

df["Open"] = pd.DataFrame(data=btc_min_data.groupby([pd.Grouper(key="Timestamp", freq="D")]).Open.first())["Open"]
df["High"] = pd.DataFrame(data=btc_min_data.groupby([pd.Grouper(key="Timestamp", freq="D")]).High.max())["High"]
df["Low"] = pd.DataFrame(data=btc_min_data.groupby([pd.Grouper(key="Timestamp", freq="D")]).Low.min())["Low"]
df["Close"] = pd.DataFrame(data=btc_min_data.groupby([pd.Grouper(key="Timestamp", freq="D")]).Close.last())["Close"]
df["Volume_BTC"] = pd.DataFrame(data=btc_min_data.groupby([pd.Grouper(key="Timestamp", freq="D")]).Volume_BTC.mean())['Volume_BTC']
df["Volume_Currency"] = pd.DataFrame(data=btc_min_data.groupby([pd.Grouper(key="Timestamp", freq="D")]).Volume_Currency.mean())["Volume_Currency"]
df["Weighted_Price"] = pd.DataFrame(data=btc_min_data.groupby([pd.Grouper(key="Timestamp", freq="D")]).Weighted_Price.mean())["Weighted_Price"]


In [11]:
df

,Open,High,Low,Close,Volume_BTC,Volume_Currency,Weighted_Price
Timestamp,,,,,,,
2011-12-31,4.39,4.58,4.39,4.58,23.829470,106.330084,4.471603
2012-01-01,4.58,5.00,4.58,5.00,7.200667,35.259720,4.806667
2012-01-02,5.00,5.00,5.00,5.00,19.048000,95.240000,5.000000
2012-01-03,5.32,5.32,5.14,5.29,11.004660,58.100651,5.252500
2012-01-04,4.93,5.57,4.93,5.57,11.914807,63.119577,5.208159
...,...,...,...,...,...,...,...
2021-03-27,55081.26,56686.15,53948.35,55839.42,1.823877,100884.732367,55193.357260
2021-03-28,55817.85,56573.04,54677.51,55790.92,1.447939,80632.115263,55832.958824
2021-03-29,55790.28,58402.68,54892.42,57600.10,3.732887,213754.555988,56913.993819
